In [29]:
import pandas as pd
import numpy as np
from statsmodels.sandbox.regression.example_kernridge import index

rng=np.random.default_rng(seed=12345)

In [30]:
# Reshaping by pivoting DataFrame objects

In [31]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),index=pd.Index(['Ohio','Colorado'],name='state'),columns=pd.Index(['one','two','three'],name='number'))

In [32]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [33]:
result = data.stack()

In [34]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [35]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [36]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [37]:
result.unstack(level="state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [38]:
s1 = pd.Series([0,1,2,3],index=['a','b','c','d'],dtype="Int64")

In [39]:
s2=pd.Series([4,5,6],index=['c','d','e'],dtype="Int64")

In [40]:
data2 = pd.concat([s1,s2],keys=['one','two'])

In [41]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [42]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [43]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [44]:
data2.unstack().stack(dropna=False)

/var/folders/rh/4jm75bkx7z55jq443q6pw8ym0000gn/T/ipykernel_4706/1518106626.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data2.unstack().stack(dropna=False)


one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

In [45]:
df = pd.DataFrame({'left':result,'right':result+5},columns=pd.Index(['left','right'],name='side'))

In [46]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [47]:
df.unstack(level='state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [48]:
df.unstack(level='state').stack(level='side')

/var/folders/rh/4jm75bkx7z55jq443q6pw8ym0000gn/T/ipykernel_4706/3112879545.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.unstack(level='state').stack(level='side')


state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

In [49]:
# Pivoting “long” to “wide” format

In [50]:
data = pd.read_csv('pandasexamples/examples/macrodata.csv')

In [51]:
data = data.loc[:,['year','quarter','realgdp','infl','unemp']]

In [52]:
data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


In [53]:
periods = pd.PeriodIndex(year=data.pop("year"),quarter=data.pop("quarter"),name='date')

/var/folders/rh/4jm75bkx7z55jq443q6pw8ym0000gn/T/ipykernel_4706/252333062.py:1: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  periods = pd.PeriodIndex(year=data.pop("year"),quarter=data.pop("quarter"),name='date')


In [54]:
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [55]:
data.index = periods.to_timestamp('D')

In [56]:
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [57]:
data = data.reindex(columns=['realgdp','infl','unemp'])

In [58]:
data.columns.name='item'

In [59]:
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [60]:
long_data = data.stack().reset_index().rename(columns={0:'value'})

In [61]:
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


In [62]:
pivoted = long_data.pivot(index='date',columns='item',values='value')

In [63]:
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [64]:
long_data['value2'] = np.random.standard_normal(len(long_data))

In [65]:
long_data[:10]

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,-0.861360
1,1959-01-01,infl,0.000,0.151947
2,1959-01-01,unemp,5.800,-1.124564
3,1959-04-01,realgdp,2778.801,0.539225
4,1959-04-01,infl,2.340,0.547563
5,1959-04-01,unemp,5.100,0.330972
6,1959-07-01,realgdp,2775.488,0.641865
7,1959-07-01,infl,2.740,-0.603929
8,1959-07-01,unemp,5.300,-0.071235
9,1959-10-01,realgdp,2785.204,0.010362


In [66]:
pivoted = long_data.pivot(index='date',columns='item')

In [67]:
pivoted.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.151947 -0.861360 -1.124564
1959-04-01  2.34  2778.801   5.1  0.547563  0.539225  0.330972
1959-07-01  2.74  2775.488   5.3 -0.603929  0.641865 -0.071235
1959-10-01  0.27  2785.204   5.6 -0.042084  0.010362  0.169003
1960-01-01  2.31  2847.699   5.2  0.211652 -0.470311 -0.021444

In [68]:
pivoted['value'][:10]

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2
1960-04-01,0.14,2834.390,5.2
1960-07-01,2.70,2839.022,5.6
1960-10-01,1.21,2802.616,6.3
1961-01-01,-0.40,2819.264,6.8


In [ ]:
# Pivoting “wide” to “long” format

In [69]:
df = pd.DataFrame({'key':['foo','bar','baz'],'A':[1,2,3],'B':[4,5,6],'C':[7,8,9]})

In [70]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [71]:
melted = pd.melt(df,id_vars='key')

In [72]:
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [73]:
reshaped = melted.pivot(index='key',columns='variable',values='value')

In [74]:
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [75]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [76]:
pd.melt(df,id_vars='key',value_vars=['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [77]:
pd.melt(df,value_vars=['A','B','C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [78]:
pd.melt(df,value_vars=['key','A','B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


In [79]:
pd.melt(df,value_vars=['key','A','B'],id_vars='key')

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
